In [11]:
import pygame
import tools as tl
import numpy as np
import scipy as sp
import random
from copy import deepcopy


# Variables you can read:

`self.position`
- this bird's position

`self.velocity`
- this bird's velocity

`self.mouseDown`
- is true if the mouse is held down and false if it is released

`self.mousePos`
- gives the position of the mouse cursor

# Useful expressions:

`for bird in flock`
- iterates over every bird in a given flock, where flock is a collection of birds

# Functions you have access to:

`self.boundary.periodicDisplacement([position_a], [position_b])`
- gives the direction vector pointing from position_b to position_a

`[vector].normalize()`
- gives the normalized form of the given vector

`[vector].normalize_ip()`
- computes the normalized form of the given vector and saves it back into vector (normalize in-place)

`limit([vector], [max_length])`
- shortens any vector to be at most max_length long


# Set Rule Weights

Here we define the variables that control our birds' behaviour.

`*Weight` all control how strong each sub-behaviour is (i.e. how much influence this has on the bird's behaviour). For example, if you want the birds to flock closer together, you can increase the `cohesionWeight`.

`*Radius` all control how far a bird will see other birds for a given behaviour. For example, if you want a bird to only avoid other birds within a smaller radius, you can lower the `separationRadius`.

Once you've defined each behaviour, feel free to play around with these variables!

In [12]:
def _setRuleWeights(self):
    self.maxSpeed = 5
    self.maxAcceleration = 1


    self.alignmentWeight = 0.025
    self.cohesionWeight = 0.1
    self.separationWeight = 0.2
    self.mouseFollowWeight = 0.2
    self.randomPerturbationWeight = 0.1

    self.alignmentRadius = 70
    self.cohesionRadius = 100
    self.separationRadius = 50
    self.mouseFollowRadius = 200

def limit(vector, length):
    tmp = deepcopy(vector)
    if tmp.length() > length:
        tmp.normalize_ip()
        tmp *= length
    return tmp

# Rule Cohesion

Here we want the birds to fly towards the average position of nearby birds. This will encourage "flocking" behaviour where birds will coalesce into large groups.

The general idea here would be to look at every bird that's nearby (i.e. within the `cohesionRadius`), then take the average position of those birds within the radius, and use that to update our bird. Remember that we want to find an acceleration that corrects our bird's velocity to fly *towards* the average position.

In [13]:
def _ruleCohesion(self, flock):

    acceleration=pygame.Vector2(0, 0)
    total_positions=pygame.Vector2(0,0)
    
    acceleration+=self.position
    number_of_birds_in_flocks=0

    for bird in flock:
        if bird==self:
            continue

        
        displacement = self.boundary.periodicDisplacement(self.position,bird.position)
        distance=displacement.length()

        if (distance) <= float(self.cohesionRadius):
            total_positions += (bird.position)
            number_of_birds_in_flocks+=1


    if number_of_birds_in_flocks==0:
        return acceleration

    total_positions/=number_of_birds_in_flocks    
    acceleration=total_positions-(self.position+self.velocity)

    acceleration.normalize_ip()
    acceleration*=self.maxSpeed
    acceleration-=self.velocity
    acceleration=limit(acceleration,self.maxAcceleration)

    return acceleration
    

    # This method below didn't work as it paired two birds together instead of making a flock
    #Find the shortest distance for the bird
    '''for bird in flock:
        if bird==self:
            continue

        dis =self.boundary.periodicDisplacement(self.position,bird.position)
        distance=dis.length()

        if distance<shortest_distance:
            shortest_distance=distance
            bird_b_pos=bird.position
    
    return (bird.position - self.position+self.velocity)'''


# Rule Alignment

We want to teach the birds to fly in a direction that matches the average direction of nearby birds. This will make sure that our birds will fly *together* as a group (think about how migrating flocks will all travel in the exact same direction).

Similar to the cohesion rule, the general idea here would be to look at every bird that's nearby (i.e. within the `alignmentRadius`), then take the average velocities of those birds within the radius, and use that to update our bird. Remember that we want to find an acceleration that corrects our bird's velocity towards the average velocity of the birds!

In [14]:
def _ruleAlignment(self, flock):
    acceleration = pygame.Vector2(0, 0)

    total_displacements=pygame.Vector2(0,0)

    total_displacements+=self.position
    number_of_birds_in_flocks=0

    for bird in flock:
        if bird==self:
            continue

        displacement=self.boundary.periodicDisplacement(self.position,bird.position)
        distance=displacement.length()
        displacement/=distance*distance

        if (distance) <= (self.alignmentRadius):
            acceleration+=bird.velocity.normalize()
            number_of_birds_in_flocks+=1

       
        
    if number_of_birds_in_flocks==0:
        return acceleration
        
    acceleration/=number_of_birds_in_flocks
    acceleration.normalize_ip()
    acceleration*=self.maxSpeed
    acceleration-=self.velocity
    acceleration=limit(acceleration,self.maxAcceleration)
    return acceleration

# Rule Separation

We want to make sure that birds will avoid getting too close to other birds (otherwise they might collide!). 

We therefore want to look at any bird within the `separationRadius` and make sure that we want to fly away from those birds.

In [15]:
def _ruleSeparation(self, flock):
    acceleration = pygame.Vector2(0, 0)

    total_displacements=pygame.Vector2(0,0)

    total_displacements+=self.position
    number_of_birds_in_flocks=0

    for bird in flock:
        if bird==self:
            continue

        displacement=self.boundary.periodicDisplacement(self.position,bird.position)
        distance=displacement.length()
        displacement/=distance*distance

        if (distance) <= (self.separationRadius):
            acceleration+=displacement
            number_of_birds_in_flocks+=1

       
        
    if number_of_birds_in_flocks==0:
        return acceleration
        
    acceleration/=number_of_birds_in_flocks
    acceleration.normalize_ip()
    acceleration*=self.maxSpeed
    acceleration-=self.velocity
    acceleration=limit(acceleration,self.maxAcceleration)
    return acceleration

# Rule Random

To make things more interesting, we can add a small random perturbation since each bird does have a mind of its own and could decide to deviate slightly from what the flock in general is doing.

In [16]:
def _ruleRandomPerturbation(self):
    acceleration = pygame.Vector2((np.random.uniform(-1,1)), (np.random.uniform(-1,1)))

    random_number=random.randint(1,100)


    acceleration-=(self.position+self.velocity)

    acceleration.normalize_ip()
    acceleration*=self.maxSpeed
    acceleration-=self.velocity
    acceleration=limit(acceleration,self.maxAcceleration)

    return acceleration

# Rule Mouse

And to make our flock a little bit more interactive, we give an extra rule to aim the flock towards wherever the mouse is held down at. This will look very similar to the cohesion rule, we're just adding an input to make the birds fly towards the mouse cursor (`mousePos` gives the position) if it is held down (`mouseDown` is true).

In [17]:
def _ruleMouseFollow(self):
    acceleration = pygame.Vector2(0, 0)

    if self.mouseDown==True:

        acceleration=self.mousePos-(self.position+self.velocity)

        acceleration.normalize_ip()
        acceleration*=self.maxSpeed
        acceleration-=self.velocity
        acceleration=limit(acceleration,self.maxAcceleration)

    return acceleration

# The Backend

Here's where all the secret code is that makes combines all the behaviours you wrote above together! Take a look if you're interested but you don't really need to touch anything past this to make the program work (but go ahead if you feel daring enough!)

In [18]:
class Bird:
    def __init__(self, x, y):
        # init position
        self.position = pygame.Vector2(x, y)
        # init velocity
        vx = np.random.uniform(-1, 1)
        vy = np.random.uniform(-1, 1)
        self.velocity = pygame.Vector2(vx, vy)
        self.velocity.normalize_ip()
        self.velocity *= np.random.uniform(1, 5)
        self.maxSpeed = 5
        # init acceleration
        self.acceleration = pygame.Vector2(0, 0)
        self.maxAcceleration = 1
        # init boundary
        self.boundary = Boundary(0, 800, 0, 600)
        # init display
        self.size = 3
        self.angle = 0
        self.color = (255, 255, 255)
        self.secondaryColor = (0, 0, 0)
        self.stroke = 2
        self.mouseDown = False
        self.mousePos = pygame.Vector2(0, 0)
        # init rule weights
        self.setRuleWeights()
    
    def computeAcceleration(self, flock):
        self.acceleration *= 0
        alignment = self.ruleAlignment(flock)
        self.acceleration += alignment*self.alignmentWeight
        cohesion = self.ruleCohesion(flock)
        self.acceleration += cohesion*self.cohesionWeight
        separation = self.ruleSeparation(flock)
        self.acceleration += separation*self.separationWeight
        randomPerturbation = self.ruleRandomPerturbation()
        self.acceleration += randomPerturbation*self.randomPerturbationWeight
        mouseFollow = self.ruleMouseFollow()
        self.acceleration += mouseFollow*self.mouseFollowWeight

    def update(self, flock):
        self.computeAcceleration(flock)

        self.velocity += self.acceleration
        self.velocity = limit(self.velocity, self.maxSpeed)
        
        self.position += self.velocity
        self.boundary.periodicProject(self.position)
        
        self.angle = np.arctan2(self.velocity.y, self.velocity.x) + np.pi/2

    def setRuleWeights(self):
        _setRuleWeights(self)

    def ruleAlignment(self, flock):
        return _ruleAlignment(self, flock)
    
    def ruleCohesion(self, flock):
        return _ruleCohesion(self, flock)
    def ruleSeparation(self, flock):
        return _ruleSeparation(self, flock)

    def ruleRandomPerturbation(self):
        return _ruleRandomPerturbation(self)

    def ruleMouseFollow(self):
        return _ruleMouseFollow(self)

    def Draw(self, screen, distance, scale):
        ps = []
        #initialize a 3x3 np array
        points = np.zeros((3, 2), dtype=int)

        # create a triangle
        points[0,:] = np.array([0,-self.size])
        points[1,:] = np.array([np.sqrt(self.size),np.sqrt(self.size)])
        points[2,:] = np.array([-np.sqrt(self.size),np.sqrt(self.size)])

        for point in points:
            rotation_matrix = np.array([[np.cos(self.angle), -np.sin(self.angle)], [np.sin(self.angle), np.cos(self.angle)]])
            rotated = np.matmul(rotation_matrix,point)

            x = int(rotated[0] * scale) + self.position.x
            y = int(rotated[1] * scale) + self.position.y
            ps.append((x, y))

        pygame.draw.polygon(screen, self.secondaryColor, ps)
        pygame.draw.polygon(screen, self.color, ps, self.stroke) 

In [19]:
class Boundary:
    def __init__(self, min_x, max_x, min_y, max_y):
        self.min_x = min_x
        self.max_x = max_x
        self.min_y = min_y
        self.max_y = max_y
    
    def size_x(self):
        return self.max_x - self.min_x

    def size_y(self):
        return self.max_y - self.min_y
    
    def periodicProject(self, p):
        while p.x > self.max_x:
            p.x -= self.size_x()
        while p.x < self.min_x:
            p.x += self.size_x()
        while p.y > self.max_y:
            p.y -= self.size_y()
        while p.y < self.min_y:
            p.y += self.size_y()
    
    def periodicDisplacement(self, p, q):
        # gives the vector pointing from q to p, taking into account periodic boundary conditions
        displacement = p - q
        if displacement.x > self.size_x()/2:
            displacement.x -= self.size_x()
        if displacement.x < -self.size_x()/2:
            displacement.x += self.size_x()
        if displacement.y > self.size_y()/2:
            displacement.y -= self.size_y()
        if displacement.y < -self.size_y()/2:
            displacement.y += self.size_y()
        return displacement

In [20]:
Width, Height = 1920, 1080
Width = 960
Height = 540
white, black = (217, 217, 217), (12, 12, 12)
size = (Width, Height)

window = pygame.display.set_mode(size, pygame.RESIZABLE)
clock = pygame.time.Clock()
fps = 60

scale = 10
Distance = 5
speed = 0.0005

flock = []
n = 50

for i in range(n):
	flock.append(Bird(random.randint(20, Width-20), random.randint(20, Height-20)))

keyPressed = False
run = True
while run:
	clock.tick(fps)
	window.fill((10, 10, 15))

	for event in pygame.event.get():
		if event.type == pygame.QUIT:
			run = False
		if event.type == pygame.KEYDOWN:
			if event.key == pygame.K_ESCAPE:
				run = False
			keyPressed = True

	# update the window size
	Width, Height = window.get_size()

	for boid in flock:
		boid.radius = scale
		boid.boundary = Boundary(0, Width, 0, Height)
		boid.mousePos = pygame.Vector2(pygame.mouse.get_pos())
		boid.mouseDown = pygame.mouse.get_pressed()[0]
		boid.update(flock)
		boid.Draw(window, Distance, scale)

	keyPressed = False
	pygame.display.flip()
pygame.quit()